In [8]:
import time
import pandas as pd
import numpy as np
import os
from urllib.parse import quote
from selenium import webdriver

In [9]:
# Setting up the Chrome driver
chrome_driver_path = "E:\Major Project 2\IPL Player Performance Prediction\chromedriver-win64\chromedriver.exe"
webdriver_path = "E:\Major Project 2\IPL Player Performance Prediction\chromedriver-win64\Chromedriver"
os.chdir(webdriver_path)

# %cd webdriver_path

driver = webdriver.Chrome(chrome_driver_path)

C:\Users\Omkar\AppData\Local\Temp\ipykernel_14404\2703813350.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path)


In [10]:
# Extracting recent stats of the players
final_data = pd.DataFrame()  # Final dataframe to store
# players= ["RG Sharma","SA Yadav", "TH David", "Ishan Kishan", "JJ Bumrah", "D Madushanka", "HH Pandya", "NT Tilak Varma","PP Chawla", "G Coetzee", "Mohammad Nabi"]  # all the player data [MI]
# players = ['DA Warner', 'PP Shaw',  'HC Brook', 'RR Pant', 'A Nortje', 'Kuldeep Yadav', 'Mukesh Kumar', 'KK Ahmed', 'JA Richardson', 'AR Patel', 'Lalit Yadav', 'MR Marsh'] # [DC]
# players = ['D Padikkal', 'A Badoni', 'KL Rahul', 'Q de Kock', 'N Pooran', 'MA Wood', 'Mohsin Khan', 'Ravi Bishnoi', 'Yash Thakur', 'Shivam Mavi', 'DJ Hooda', 'KH Pandya', 'MP Stoinis'] # [LSG]
# players = ['B Sai Sudharsan', 'DA Miller', 'Shubman Gill', 'WP Saha', 'Mohammed Shami', 'MM Sharma', 'Noor Ahmad', 'SH Johnson', 'R Tewatia', 'Rashid Khan', 'V Shankar', 'M Shahrukh Khan'] #[GT]
# players = ['YBK Jaiswal', 'R Parag', 'SO Hetmyer', 'R Powell', 'SV Samson', 'DC Jurel', 'JC Buttler', 'YS Chahal', 'M Prasidh Krishna', 'TA Boult', 'R Ashwin'] #[RR]
players = ['S Dhawan', 'JM Sharma', 'Prabhsimran Singh', 'Arshdeep Singh', 'K Rabada', 'RD Chahar', 'HV Patel', 'LS Livingstone', 'A Taide', 'SM Curran', 'Sikandar Raza'] #[PK]



In [11]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Looping through all the players
for i in players[0:]: 
   # Encode the player name for URL
    encoded_player_name = quote(i)
    
    # Accessing the web page for the current player's stats
    url = f"http://www.cricmetric.com/playerstats.py?" \
          f"player={encoded_player_name}&role=all&format=TWENTY20&" \
          f"groupby=match&playerStatsFilters=on&start_date=2017-01-01&" \
          f"end_date=2023-05-18&start_over=0&end_over=9999"
    
    driver.get(url)
    
    # Scrolling down to load all the stats
    # driver.execute_script("window.scrollTo(0, 500)")
    driver.maximize_window()
    time.sleep(3)
    
    try:
        # Extracting batting stats of the player
        # Wait for the batting table to be present
        batting_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="TWENTY20-Batting"]/div/table'))
        )

        # batting_table = driver.find_element_by_xpath(
        #     '//*[@id="TWENTY20-Batting"]/div/table')

        bat = batting_table.text
        stats = pd.DataFrame(bat.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats.columns = stats.iloc[0]
        stats = stats[1:]
        del stats['%'] 
        stats = stats[['Match','Runs','Balls','Outs','SR',
                       '50','100','4s','6s','Dot']]
        stats.columns = ['Match','Runs Scored','Balls Played',
                         'Out','Bat SR','50','100','4s Scored',
                         '6s Scored','Bat Dot%']
        
        # driver.execute_script("arguments[0].scrollIntoView();", bowling_tab)

        # Switching to bowling stats tab
        bowling_tab = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="TWENTY20-Bowling-tab"]'))
        )
        # bowling_tab = WebDriverWait(driver, 10).until(
        #     EC.element_to_be_clickable((By.XPATH, '//*[@id="TWENTY20-Bowling-tab"]'))
        # )

        bowling_tab.click()
        time.sleep(5)
    
        # Extracting bowling stats of the player
        bowling_table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="TWENTY20-Bowling"]/div/table'))
        )
        bowl = bowling_table.text
        stats2 = pd.DataFrame(bowl.split('\n'))[0].str.split(' ',expand=True)[0:-1]
        stats2.columns = stats2.iloc[0]
        stats2 = stats2[1:]
        stats2 = stats2[['Match','Overs','Runs','Wickets','Econ',
                         'Avg','SR','5W','4s','6s','Dot%']]
        stats2.columns = ['Match','Overs Bowled','Runs Given',
                          'Wickets Taken','Econ','Bowl Avg',
                          'Bowl SR','5W','4s Given','6s Given',
                          'Bowl Dot%']
        
    except:
        # If stats for current player not found,
        # create empty dataframe
        # print("I m in except")
        stats2 = pd.DataFrame({'Match':pd.Series(stats['Match'][0:1]),
                               'Overs Bowled':[0],'Runs Given':[0],
                               'Wickets Taken':[0],'Econ':[0],
                               'Bowl Avg':[0],'Bowl SR':[0],'5W':[0],
                               '4s Given':[0],'6s Given':[0],
                               'Bowl Dot%':[0]})
        
    # Merge batting and bowling stats
    merged_stats = pd.merge(stats,stats2,on='Match',how='outer').fillna(0)
    # print(merged_stats)
    merged_stats = merged_stats.sort_values(by=['Match'])
    
    # Create lagged variables for future performance prediction
    merged_stats.insert(loc=0, column='Player', value=i)
    merged_stats['next_runs'] = merged_stats['Runs Scored'].shift(-1)
    merged_stats['next_balls'] = merged_stats['Balls Played'].shift(-1)
    merged_stats['next_overs'] = merged_stats['Overs Bowled'].shift(-1)
    merged_stats['next_runs_given'] = merged_stats['Runs Given'].shift(-1)
    merged_stats['next_wkts'] = merged_stats['Wickets Taken'].shift(-1)
    final_data = final_data.append(merged_stats)
    
final_data = final_data[final_data['Match']!=0]

final_data['Bowl Avg'] = np.where(final_data['Bowl Avg']=='-',
                                 0,final_data['Bowl Avg'])
final_data['Bowl SR'] = np.where(final_data['Bowl SR']=='-',
                                0,final_data['Bowl SR'])
final_data = final_data[['Player','Match', 'Runs Scored',
                         'Balls Played', 'Out', 'Bat SR',
                         '50', '100', '4s Scored',
                         '6s Scored','Bat Dot%',
                         'Overs Bowled','Runs Given',
                         'Wickets Taken', 'Econ',
                         'Bowl Avg', 'Bowl SR', '5W',
                         '4s Given', '6s Given',
                         'Bowl Dot%', 'next_runs',
                         'next_balls', 'next_overs',
                         'next_runs_given', 'next_wkts']]
final_data = final_data.replace('-',0)
final_data = final_data.fillna(0)
# print(final_data)
final_data
# final_data.to_csv('E:/Major Project 2/IPL Player Performance Prediction/output_file.csv', index=False)


C:\Users\Omkar\AppData\Local\Temp\ipykernel_14404\1880282639.py:95: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append(merged_stats)
C:\Users\Omkar\AppData\Local\Temp\ipykernel_14404\1880282639.py:95: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append(merged_stats)
C:\Users\Omkar\AppData\Local\Temp\ipykernel_14404\1880282639.py:76: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  stats2 = pd.DataFrame({'Match':pd.Series(stats['Match'][0:1]),
C:\Users\Omkar\AppData\Local\Temp\ipykernel_14404\1880282639.py:95: F

,Player,Match,Runs Scored,Balls Played,Out,Bat SR,50,100,4s Scored,6s Scored,...,Bowl SR,5W,4s Given,6s Given,Bowl Dot%,next_runs,next_balls,next_overs,next_runs_given,next_wkts
0,YBK Jaiswal,ipl2020_04,6,6,1,100.0,0,0,1,0,...,0,0,0,0,0,0,2,0,0,0
1,YBK Jaiswal,ipl2020_20,0,2,1,0.0,0,0,0,0,...,0,0,0,0,0,34,36,0,0,0
2,YBK Jaiswal,ipl2020_23,34,36,1,94.4,0,0,1,2,...,0,0,0,0,0,22,17,0,0,0
3,YBK Jaiswal,ipl2021_18,22,17,1,129.4,0,0,5,0,...,0,0,0,0,0,32,20,0,0,0
4,YBK Jaiswal,ipl2021_24,32,20,1,160.0,0,0,2,2,...,0,0,0,0,0,12,13,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,R Ashwin,tnpl2019_14,0,0,0,0,0,0,0,0,...,24.0,0,1,2,29.2,1,3,4.0,33,0
49,R Ashwin,tnpl2019_20,1,3,1,33.3,0,0,0,0,...,0,0,0,2,37.5,8,17,4.0,21,1
50,R Ashwin,tnpl2019_23,8,17,1,47.1,0,0,0,0,...,24.0,0,1,1,62.5,22,22,4.0,33,1
51,R Ashwin,tnpl2019_29,22,22,1,100.0,0,0,1,0,...,24.0,0,6,0,45.8,25,29,4.0,21,1


In [12]:
import pandas as pd
import numpy as np
import time
import math
from statistics import stdev
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import scipy.stats

models = pd.DataFrame()
# players_list = ["RG Sharma","SA Yadav", "TH David", "Ishan Kishan", "JJ Bumrah", "D Madushanka", "HH Pandya", "NT Tilak Varma","PP Chawla", "G Coetzee", "Mohammad Nabi"] #[MI]
# players_list = ['DA Warner', 'PP Shaw',  'HC Brook', 'RR Pant', 'A Nortje', 'Kuldeep Yadav', 'Mukesh Kumar', 'KK Ahmed', 'JA Richardson', 'AR Patel', 'Lalit Yadav', 'MR Marsh'] #[DC]
# players_list = ['D Padikkal', 'A Badoni', 'KL Rahul', 'Q de Kock', 'N Pooran', 'MA Wood', 'Mohsin Khan', 'Ravi Bishnoi', 'Yash Thakur', 'Shivam Mavi', 'DJ Hooda', 'KH Pandya', 'MP Stoinis'] # [LSG]
# players_list = ['B Sai Sudharsan', 'DA Miller', 'Shubman Gill', 'WP Saha', 'Mohammed Shami', 'MM Sharma', 'Noor Ahmad', 'SH Johnson', 'R Tewatia', 'Rashid Khan', 'V Shankar', 'M Shahrukh Khan'] #[GT]
# players_list = ['YBK Jaiswal', 'R Parag', 'SO Hetmyer', 'R Powell', 'SV Samson', 'DC Jurel', 'JC Buttler', 'YS Chahal', 'M Prasidh Krishna', 'TA Boult', 'R Ashwin'] #[RR]
players_list = ['S Dhawan', 'JM Sharma', 'Prabhsimran Singh', 'Arshdeep Singh', 'K Rabada', 'RD Chahar', 'HV Patel', 'LS Livingstone', 'A Taide', 'SM Curran', 'Sikandar Raza'] #[PK]

for player_name in players_list:
    # Filter the data for the current player
    player_data = final_data[final_data['Player'] == player_name]

    # Remove rows with missing values
    player_new = player_data.dropna()
    
    # player.to_csv('E:/Major Project 2/IPL Player Performance Prediction/player_new.csv', index=False)

    # Predict next runs
    X_runs = player_new[player_new.columns[2:11]]
    y_runs = player_new[player_new.columns[21:22]]
    # print(X_runs)
    # print("*****")
    # print(y_runs)
    X_train_runs, X_test_runs, y_train_runs,y_test_runs = train_test_split(X_runs, y_runs,random_state=123)
    ridge_runs = pd.DataFrame()

    # print("*****")
    # print(X_train_runs)
    # print("*****")
    # print(X_test_runs)
    # print("*****")
    # print(y_train_runs)
    # print("*****")
    # print(y_test_runs)

    # Iterate over a range of alpha values
    for j in range(0, 101):
        points_runs = linear_model.Ridge(alpha=j).fit(X_train_runs,y_train_runs)
        ridge_df_runs = pd.DataFrame({'Alpha': pd.Series(j),'Train': pd.Series(points_runs.score(X_train_runs,y_train_runs)), 'Test': pd.Series(points_runs.score(X_test_runs, y_test_runs))})
        ridge_runs = ridge_runs.append(ridge_df_runs)

    # Calculate average score
    ridge_runs['Average'] = ridge_runs[['Train', 'Test']].mean(axis=1)
    # print("Ridge****")
    # print(ridge_runs)
    try:
        # Find the alpha value with the highest average score
        k_runs = ridge_runs[ridge_runs['Average'] == ridge_runs['Average'].max()]['Alpha'][0]
        k_runs = k_runs.head(1)[0]
    except:
        k_runs = ridge_runs[ridge_runs['Average'] == ridge_runs['Average'].max()]['Alpha'][0]

    # Train the model with the best alpha value
    next_runs = linear_model.Ridge(alpha=k_runs)
    next_runs.fit(X_train_runs, y_train_runs)
    sd_next_runs = stdev(X_train_runs['Runs Scored'].astype('float'))
# --------------------------------------------------------

    # Predict next balls
    X_balls = player_new[player_new.columns[2:11]]
    y_balls = player_new[player_new.columns[22:23]]
    X_train_balls, X_test_balls, y_train_balls, y_test_balls = train_test_split(X_balls, y_balls,random_state=123)
    ridge_balls = pd.DataFrame()

    # Iterate over a range of alpha values
    for j in range(0, 101):
        points_balls = linear_model.Ridge(alpha=j).fit(X_train_balls, y_train_balls)
        ridge_df_balls = pd.DataFrame({'Alpha': pd.Series(j),'Train': pd.Series(points_balls.score(X_train_balls,y_train_balls)), 'Test': pd.Series(points_balls.score(X_test_balls, y_test_balls))})
        ridge_balls = ridge_balls.append(ridge_df_balls)

    # Calculate average score
    ridge_balls['Average'] = ridge_balls[['Train', 'Test']].mean(axis=1)

    try:
        # Find the alpha value with the highest average score
        k_balls = ridge_balls[ridge_balls['Average'] == ridge_balls['Average'].max()]['Alpha'][0]
        k_balls = k_balls.head(1)[0]
    except:
        k_balls = ridge_balls[ridge_balls['Average'] == ridge_balls['Average'].max()]['Alpha'][0]

    # Train the model with the best alpha value
    next_balls = linear_model.Ridge(alpha=k_balls)
    next_balls.fit(X_train_balls, y_train_balls)
    sd_next_balls = stdev(X_train_balls['Balls Played'].astype('float'))

    # Predict next overs
    X_overs = player_new[player_new.columns[11:21]]
    y_overs = player_new[player_new.columns[25:26]]
    X_train_overs, X_test_overs, y_train_overs, y_test_overs = train_test_split(X_overs, y_overs, random_state=123)
    ridge_overs = pd.DataFrame()

    # print(X_overs)
    # print(y_overs)

    # for i in X_overs:
    #     print(type(i))
    # for i in y_overs:
    #     print(type(i))

    # print(X_train_overs)
    # print("******")
    # print(X_test_overs)
    # print("y data")
    # print(y_train_overs)
    # print("******")
    # print(y_test_overs)

    # Remove commas and convert to float for relevant columns
    columns_to_convert = ['Runs Given']
    for col in columns_to_convert:
        X_train_overs[col] = X_train_overs[col].astype(str).str.replace(',', '').astype(float)
        X_test_overs[col] = X_test_overs[col].astype(str).str.replace(',', '').astype(float)

    # print("Datatype:****************************")
    # # print(type(X_train_overs))
    # for i in X_train_overs["Runs Given"]:
    #     print(i, end="")
    #     print(type(i))
    # print("Datatype:****************************")

    # Iterate over a range of alpha values
    for j in range(0, 101):
        points_overs = linear_model.Ridge(alpha=j).fit(X_train_overs, y_train_overs)
        ridge_df_overs = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points_overs.score(X_train_overs,y_train_overs)), 'Test': pd.Series(points_overs.score(X_test_overs, y_test_overs))})
        ridge_overs = ridge_overs.append(ridge_df_overs)

# -------------------------------------------
    # Calculate average score
    ridge_overs['Average'] = ridge_overs[['Train', 'Test']].mean(axis=1)

    try:
        # Find the alpha value with the highest average score
        k_overs = ridge_overs[ridge_overs['Average'] == ridge_overs['Average'].max()]['Alpha'][0]
        k_overs = k_overs.head(1)[0]
    except:
        k_overs = ridge_overs[ridge_overs['Average'] == ridge_overs['Average'].max()]['Alpha'][0]

    # Train the model with the best alpha value
    next_overs = linear_model.Ridge(alpha=k_overs)
    next_overs.fit(X_train_overs, y_train_overs)
    sd_next_overs = stdev(X_train_overs['Overs Bowled'].astype('float'))

    # Predict next runs given
    X_runs_given = player_new[player_new.columns[11:21]]
    y_runs_given = player_new[player_new.columns[24:25]]
    X_train_runs_given, X_test_runs_given, y_train_runs_given, y_test_runs_given = train_test_split(X_runs_given, y_runs_given, random_state=123)
    ridge_runs_given = pd.DataFrame()

    # Remove commas and convert to float for relevant columns
    columns_to_convert = ['Runs Given']
    for col in columns_to_convert:
        X_train_runs_given[col] = X_train_runs_given[col].astype(str).str.replace(',', '').astype(float)
        X_test_runs_given[col] = X_test_runs_given[col].astype(str).str.replace(',', '').astype(float)

    columns_to_convert = ['next_runs_given']
    for col in columns_to_convert:
        y_train_runs_given[col] = y_train_runs_given[col].astype(str).str.replace(',', '').astype(float)
        y_test_runs_given[col] = y_test_runs_given[col].astype(str).str.replace(',', '').astype(float)

    # Iterate over a range of alpha values
    for j in range(0, 101):
        points_runs_given = linear_model.Ridge(alpha=j).fit(X_train_runs_given, y_train_runs_given)
        ridge_df_runs_given = pd.DataFrame({'Alpha': pd.Series(j),'Train': pd.Series(points_runs_given.score(X_train_runs_given, y_train_runs_given)), 'Test': pd.Series(points_runs_given.score(X_test_runs_given,y_test_runs_given))})
        ridge_runs_given = ridge_runs_given.append(ridge_df_runs_given)
# ----------------------------



# -----------------------------
    # # Calculate average score
    ridge_runs_given['Average'] = ridge_runs_given[['Train', 'Test']].mean(axis=1)

    try:
        # Find the alpha value with the highest average score
        k_runs_given = ridge_runs_given[ridge_runs_given['Average'] == ridge_runs_given['Average'].max()]['Alpha'][0]
        k_runs_given = k_runs_given.head(1)[0]
    except:
        k_runs_given = ridge_runs_given[ridge_runs_given['Average'] == ridge_runs_given['Average'].max()]['Alpha'][0]

    # Train the model with the best alpha value
    next_runs_given = linear_model.Ridge(alpha=k_runs_given)
    next_runs_given.fit(X_train_runs_given, y_train_runs_given)
    sd_next_runs_given = stdev(X_train_runs_given['Runs Given'].astype('float'))

# -------------------------------------
    # Predict next wickets
    X_wkts = player_new[player_new.columns[11:21]]
    y_wkts = player_new[player_new.columns[23:24]]
    X_train_wkts, X_test_wkts, y_train_wkts, y_test_wkts = train_test_split(X_wkts, y_wkts, random_state=123)
    ridge_wkts = pd.DataFrame()

    # Remove commas and convert to float for relevant columns
    columns_to_convert = ['Wickets Taken','Runs Given']
    for col in columns_to_convert:
        X_train_wkts[col] = X_train_wkts[col].astype(str).str.replace(',', '').astype(float)
        X_test_wkts[col] = X_test_wkts[col].astype(str).str.replace(',', '').astype(float)

    # Iterate over a range of alpha values
    for j in range(0, 101):
        points_wkts = linear_model.Ridge(alpha=j).fit(X_train_wkts, y_train_wkts)
        ridge_df_wkts = pd.DataFrame({'Alpha': pd.Series(j), 'Train': pd.Series(points_wkts.score(X_train_wkts, y_train_wkts)),
                                       'Test': pd.Series(points_wkts.score(X_test_wkts, y_test_wkts))})
        ridge_wkts = ridge_wkts.append(ridge_df_wkts)

    # Calculate average score
    ridge_wkts['Average'] = ridge_wkts[['Train', 'Test']].mean(axis=1)

    try:
        # Find the alpha value with the highest average score
        k_wkts = ridge_wkts[ridge_wkts['Average'] == ridge_wkts['Average'].max()]['Alpha'][0]
        k_wkts = k_wkts.head(1)[0]
    except:
        k_wkts = ridge_wkts[ridge_wkts['Average'] == ridge_wkts['Average'].max()]['Alpha'][0]

    # Train the model with the best alpha value
    next_wkts = linear_model.Ridge(alpha=k_wkts)
    next_wkts.fit(X_train_wkts, y_train_wkts)
    sd_next_wkts = stdev(X_train_wkts['Wickets Taken'].astype('float'))
# -------------------------------------

    player = player_new
    # Remove commas and convert to float for relevant columns
    columns_to_convert = ['Runs Scored', 'Runs Given', 'next_runs_given', 'next_runs', 'next_balls', 'next_wkts']
    for col in columns_to_convert:
        player[col] = player[col].astype(str).str.replace(',', '').astype(float)

    # Get the latest data for the player
    latest = player.groupby('Player').tail(1)
    # latest = player.groupby('Player')

    # Predict next runs, balls, overs, runs given, and wickets
    latest['next_runs'] = next_runs.predict(latest[latest.columns[2:11]])
    latest['next_balls'] = next_balls.predict(latest[latest.columns[2:11]])
    latest['next_overs'] = next_overs.predict(latest[latest.columns[11:21]])
    latest['next_runs_given'] = next_runs_given.predict(latest[latest.columns[11:21]])
    latest['next_wkts'] = next_wkts.predict(latest[latest.columns[11:21]])

    # Calculate confidence intervals for each prediction
    latest['next_runs_ll_95'], latest['next_runs_ul_95'] = latest['next_runs'] - scipy.stats.norm.ppf(.95) * (sd_next_runs / math.sqrt(len(X_train_runs))),latest['next_runs'] + scipy.stats.norm.ppf(.95) * (sd_next_runs / math.sqrt(len(X_train_runs)))
    latest['next_balls_ll_95'], latest['next_balls_ul_95'] = latest['next_balls'] - scipy.stats.norm.ppf(.95) * (sd_next_balls / math.sqrt(len(X_train_balls))),latest['next_balls'] + scipy.stats.norm.ppf(.95) * (sd_next_balls / math.sqrt(len(X_train_balls)))
    latest['next_overs_ll_95'], latest['next_overs_ul_95'] = latest['next_overs'] - scipy.stats.norm.ppf(.95) * (sd_next_overs / math.sqrt(len(X_train_overs))), latest['next_overs'] + scipy.stats.norm.ppf(.95) * (sd_next_overs / math.sqrt(len(X_train_overs)))
    latest['next_runs_given_ll_95'], latest['next_runs_given_ul_95'] = latest['next_runs_given'] - scipy.stats.norm.ppf(.95) * (sd_next_runs_given / math.sqrt(len(X_train_runs_given))), latest['next_runs_given'] + scipy.stats.norm.ppf(.95) * (sd_next_runs_given / math.sqrt(len(X_train_runs_given)))
    latest['next_wkts_ll_95'], latest['next_wkts_ul_95'] = latest['next_wkts'] - scipy.stats.norm.ppf(.95) * (sd_next_wkts / math.sqrt(len(X_train_wkts))),latest['next_wkts'] + scipy.stats.norm.ppf(.95) * (sd_next_wkts / math.sqrt(len(X_train_wkts)))

    # Append the latest predictions to the models dataframe
    models = models.append(latest)
    models
    # models.to_csv('E:/Major Project 2/IPL Player Performance Prediction/models.csv', index=False)

c:\Users\Omkar\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\Omkar\AppData\Local\Temp\ipykernel_14404\3152155871.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ridge_runs = ridge_runs.append(ridge_df_runs)
C:\Users\Omkar\AppData\Local\Temp\ipykernel_14404\3152155871.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ridge_runs = ridge_runs.append(ridge_df_runs)
C:\Users\Omkar\AppData\Local\Temp\ipykernel_14404\3152155871.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ridge_runs = rid

In [13]:
# Adjusting values based on conditions and rounding

# Adjusting next_runs_given based on next_overs
models['next_runs_given'] = np.where(
    models['next_overs'] > 4,
    models['next_runs_given'] / models['next_overs'] * 4,
    models['next_runs_given']
)
models['next_runs_given_ll_95'] = np.where(
    models['next_overs'] > 4,
    models['next_runs_given_ll_95'] / models['next_overs'] * 4,
    models['next_runs_given_ll_95']
)
models['next_runs_given_ul_95'] = np.where(
    models['next_overs'] > 4,
    models['next_runs_given_ul_95'] / models['next_overs'] * 4,
    models['next_runs_given_ul_95']
)

# Limiting next_overs to a maximum of 4
models['next_overs'] = np.where(
    models['next_overs'] > 4,
    4,
    models['next_overs']
)
models['next_overs_ll_95'] = np.where(
    models['next_overs_ll_95'] > 4,
    4,
    models['next_overs_ll_95']
)
models['next_overs_ul_95'] = np.where(
    models['next_overs_ul_95'] > 4,
    4,
    models['next_overs_ul_95']
)

# Adjusting next_runs based on next_balls
models['next_runs'] = np.where(
    models['next_balls'] < 0,
    0,
    models['next_runs']
)
models['next_runs_ll_95'] = np.where(
    models['next_balls'] < 0,
    0,
    models['next_runs_ll_95']
)
models['next_runs_ul_95'] = np.where(
    models['next_balls'] < 0,
    0,
    models['next_runs_ul_95']
)

# Setting next_runs to a minimum of 1
models['next_runs'] = np.where(
    models['next_runs'] < 0,
    1,
    models['next_runs']
)
models['next_runs_ll_95'] = np.where(
    models['next_runs_ll_95'] < 0,
    1,
    models['next_runs_ll_95']
)
models['next_runs_ul_95'] = np.where(
    models['next_runs_ul_95'] < 0,
    1,
    models['next_runs_ul_95']
)

# Adjusting next_runs based on next_balls if next_balls > 100
models['next_runs'] = np.where(
    models['next_balls'] > 100,
    models['next_runs'] / models['next_balls'] * 5,
    models['next_runs']
)
models['next_runs_ll_95'] = np.where(
    models['next_balls'] > 100,
    models['next_runs_ll_95'] / models['next_balls'] * 5,
    models['next_runs_ll_95']
)
models['next_runs_ul_95'] = np.where(
    models['next_balls'] > 100,
    models['next_runs_ul_95'] / models['next_balls'] * 5,
    models['next_runs_ul_95']
)

# Limiting next_balls to a maximum of 5
models['next_balls'] = np.where(
    models['next_balls'] > 100,
    5,
    models['next_balls']
)
models['next_balls_ll_95'] = np.where(
    models['next_balls_ll_95'] > 100,
    5,
    models['next_balls_ll_95']
)
models['next_balls_ul_95'] = np.where(
    models['next_balls_ul_95'] > 100,
    5,
    models['next_balls_ul_95']
)

# Setting next_balls to a minimum of 1
models['next_balls'] = np.where(
    models['next_balls'] < 0,
    1,
    models['next_balls']
)
models['next_balls_ll_95'] = np.where(
    models['next_balls_ll_95'] < 0,
    1,
    models['next_balls_ll_95']
)
models['next_balls_ul_95'] = np.where(
    models['next_balls_ul_95'] < 0,
    1,
    models['next_balls_ul_95']
)

# Setting next_wkts to a minimum of 1
models['next_wkts'] = np.where(
    models['next_wkts'] < 0,
    1,
    models['next_wkts']
)
models['next_wkts_ll_95'] = np.where(
    models['next_wkts_ll_95'] < 0,
    1,
    models['next_wkts_ll_95']
)
models['next_wkts_ul_95'] = np.where(
    models['next_wkts_ul_95'] < 0,
    1,
    models['next_wkts_ul_95']
)

# Rounding values to 0 decimal places
models['next_runs'] = round(models['next_runs'], 0)
models['next_runs_ll_95'] = round(models['next_runs_ll_95'], 0)
models['next_runs_ul_95'] = round(models['next_runs_ul_95'], 0)

models['next_balls'] = round(models['next_balls'], 0)
models['next_balls_ll_95'] = round(models['next_balls_ll_95'], 0)
models['next_balls_ul_95'] = round(models['next_balls_ul_95'], 0)

models['next_wkts'] = round(models['next_wkts'], 0)
models['next_wkts_ll_95'] = round(models['next_wkts_ll_95'], 0)
models['next_wkts_ul_95'] = round(models['next_wkts_ul_95'], 0)

models['next_runs_given'] = round(models['next_runs_given'], 0)
models['next_runs_given_ll_95'] = round(models['next_runs_given_ll_95'], 0)
models['next_runs_given_ul_95'] = round(models['next_runs_given_ul_95'], 0)

models['next_overs'] = round(models['next_overs'], 0)
models['next_overs_ll_95'] = round(models['next_overs_ll_95'], 0)
models['next_overs_ul_95'] = round(models['next_overs_ul_95'], 0)

# print(models)
models.to_csv('E:/Major Project 2/IPL Player Performance Prediction/models_PK.csv', index=False)